<a name='1'></a>
# 1 - Importing libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import os
import numpy as np
from numpy import genfromtxt
import PIL

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<a name='2'></a>
# 2 - Loading Pretrained Facenet Model

In [3]:
from tensorflow.keras.models import model_from_json
model = tf.keras.models.load_model('/Users/harshsingh/Desktop/face/model')
FRmodel = model

In [4]:
print(model.inputs)
print(model.outputs)

[<KerasTensor: shape=(None, 160, 160, 3) dtype=float32 (created by layer 'input_1')>]
[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'Bottleneck_BatchNorm')>]


<a name='4'></a>
# 4 - Function to preprocess images and predict them

In [5]:
def img_to_encoding(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
    img = np.around(np.array(img) / 255.0, decimals=12)
    x_train = np.expand_dims(img, axis=0) # add a dimension of 1 as first dimension
    embedding = model.predict_on_batch(x_train)
    return embedding / np.linalg.norm(embedding, ord=2)

<a name='5'></a>
# 5 - Simulation of a Database

In [6]:
database = {}
database["Harshil"] = img_to_encoding("/Users/harshsingh/Desktop/face/images/Harshil/Harshil_closeup.jpeg", FRmodel)
database["Harsh"] = img_to_encoding("/Users/harshsingh/Desktop/face/images/Harsh/Harsh_abhi.jpeg", FRmodel)
database["Aryan"] = img_to_encoding("/Users/harshsingh/Desktop/face/images/Aryan/Aryan_closeup.jpeg", FRmodel)
database["Sarang"] = img_to_encoding("/Users/harshsingh/Desktop/face/images/Sarang/Sarang_closeup.jpeg", FRmodel)
database["Papa"] = img_to_encoding("/Users/harshsingh/Desktop/face/images/Papa/Papa_plane.jpeg", FRmodel)

Load the images of Danielle and Kian:

<a name='6'></a>
# 6 - Face Verification

In [7]:
def verify(image_path, identity, database, model):
    
    encoding = img_to_encoding(image_path, model)
    dist = np.linalg.norm(tf.subtract(database[identity], encoding))
    if dist < 0.9:
        print("It's " + str(identity) + ", welcome in!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False
    print(dist)
    return dist, door_open

In [8]:
peoples = ["Harsh", "Harshil","Aryan", "Sarang", "Papa"] ### "Papa", "Saran", "Sarang"]

In [78]:
base_dir = "images"  # Path to the "images" folder
for person in peoples:
    person_folder = os.path.join(base_dir, person)
    
    # Check if the folder exists
    if os.path.exists(person_folder):
        # Iterate through each image in the person's folder
        for image_name in os.listdir(person_folder):
            image_path = os.path.join(person_folder, image_name)
            
            # Check if it's an image file (optional check based on file extension)
            if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                # Call the verify function for each image
                verify(image_path, "Papa", database, FRmodel)
            else:
                print(f"Skipping non-image file: {image_name}")


It's not Papa, please go away
1.3494343
It's not Papa, please go away
1.3059747
It's not Papa, please go away
1.0922953
It's not Papa, please go away
1.179243
It's not Papa, please go away
1.3380356
It's not Papa, please go away
1.1703198
It's not Papa, please go away
1.3531189
It's not Papa, please go away
1.2834737
It's not Papa, please go away
1.382893
It's not Papa, please go away
1.4386061
It's not Papa, please go away
1.4384489
It's not Papa, please go away
1.4638108
It's not Papa, please go away
1.3239197
It's not Papa, please go away
1.3198936
It's not Papa, please go away
1.402802
It's not Papa, please go away
1.485149
It's not Papa, please go away
1.4150637
It's Papa, welcome in!
0.0
It's Papa, welcome in!
0.6383572
It's Papa, welcome in!
0.55083877
It's Papa, welcome in!
0.5994069


In [73]:
verify("/Users/harshsingh/Desktop/face/images/Harsh/Harsh_closeup.jpeg", "Papa", database, FRmodel)

It's not Papa, please go away
1.3380356


(1.3380356, False)

<a name='7'></a>
# 7 - Face Recognition

In [9]:
def who_is_it(image_path, database, model):
    
    encoding =  img_to_encoding(image_path, model)

    min_dist = 100

    for (name, db_enc) in database.items():

        dist = np.linalg.norm(tf.subtract(db_enc, encoding))

        if dist < min_dist:
            min_dist = dist
            identity = name

    if min_dist > 0.95:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))

    return min_dist, identity

In [11]:

# Test 1 with Younes pictures
who_is_it("/Users/harshsingh/Desktop/face/data_collection_pipeline/captured_images/captured_image.jpg", database, FRmodel)

it's Harsh, the distance is 0.8697179


(0.8697179, 'Harsh')

In [47]:
for people in peoples:
      verify("/Users/harshsingh/Desktop/face/images/Harsh/Harsh_without_specs.jpeg", people, database, FRmodel)
      


It's not Harsh, please go away
1.2030718
It's not Harshil, please go away
1.2167572
It's not Aryan, please go away
1.3088598
